In [ ]:
#******************************************************************************************************************************************************#
"""
    @author Shaela Khan, Created 1st April,2022, Last Updated : 11th Aug, Thursday
#  ann.ipynb  -> In Implementation phase
#  Bird Recognition from birdsongs using Deep learning -> We are working on building a model using Deep learning techniques to identify fire and smoke from images.
#  DataSource : - https://www.kaggle.com/datasets/rtatman/british-birdsong-dataset  (Aggregated from the original Xeno-Canto Dataset.)
#  Provided dataset has a directory with - songs
#                                       - .csv file with labels
#                                       -
#  Dataset contains Image files from which, we create a train and test dataset for training purposes.
#  We then create a CNN model with possible usage of pre-trained models, that can identify the difference classes defined - this is a supervised learning
#  problem.

#  Input:  songs --> audio files xx111.wav
#  Output : The file should have
#          : Spectrograph images converted from the audio files.

"""
#*******************************************************************************************************************************************************#

In [ ]:
# Common libraries
import os,sys,random,math
from random import seed, random, randint
from tqdm import tqdm
import pathlib
from pathlib import Path

# File processing.
import csv
import pandas as pd

# Plotting and visuals.
import matplotlib.pyplot as plt, numpy as np, shutil, itertools, pickle, pandas as pd, seaborn as sn, math, time

# Stats
from scipy.spatial import distance
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from mpl_toolkits.mplot3d import Axes3D
from scipy import spatial

# Audio file processing
import librosa
import librosa.display
import soundfile as sf

# Image processing libraries
from PIL import Image

# Creating Model and model Evaluation
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# EXTRA -
%matplotlib inline

# ML, statistics
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

# Tensorflow
#from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Model, load_model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings

# EXTRA EXTRA - because they don't work upstairs?
import pydub
from pydub import AudioSegment
import IPython
from IPython.display import Audio
import os
from pydub import AudioSegment
import subprocess
warnings.filterwarnings('ignore')

# Start Code Here.
print("Hello World ! ")
print("**********  Sweet Caroline da da da !!**********")



In [ ]:
# Stupid God forsaken bug!

#path = "the path to the audio files"
#Change working directory
#os.chdir(path)
#audio_files = os.listdir()

# Don't need the number of files in the folder, just iterate over them directly using:
##for dirname, _, filenames in os.walk('./small Xeno-Canto/test/'):
    #spliting the file into the name and the extension
#    for filename in filenames:
#        name, ext = os.path.splitext(filename)
#        songname = './small Xeno-Canto/test/'+filename
#        if ext == ".mp3":
#            mp3_sound = AudioSegment.from_mp3(filename)
#            #rename them using the old name + ".wav"
#            mp3_sound.export("{0}.wav".format(name), format="wav")
import os
import pydub
from pydub import AudioSegment
# files
src_folder = "./small Xeno-Canto/songs-test"   # I think ultimately this was the fix. Because AudioSegment cannot handle relative paths?
dst_folder = "./small Xeno-Canto/songs-test-wav" # So, save the relative path as name, then try.. etc. :D

#get all music file
files = os.listdir(src_folder)
for name in files:
    #name of the file
    wav_name = name.replace(".mp3", ".wav")
    try:
        # convert wav to mp3
        sound = AudioSegment.from_mp3("{}/{}".format(src_folder, name))
        sound.export("{}/{}".format(dst_folder, wav_name), format="wav")
    except Exception as e:
        pass


print("\n Show me this is done.")


In [ ]:
# CONVERTING BIRD SONGS WHICH ARE AUDIO FILES INTO IMAGE FILES- FOR TEST DATA --> iMAGES ARE SPECTROGRAPHS OF THE AUDIO DATA.
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
pathlib.Path('img_data-test/').mkdir(parents=True, exist_ok=True)

for dirname, _, filenames in os.walk('./small Xeno-Canto/songs-test-wav/'):
    for filename in filenames:
        #print(filename)
        songname = './small Xeno-Canto/songs-test-wav/'+filename  # Because you need the whole path.
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB')
        plt.axis('off')
        base = os.path.splitext(filename)[0]
        # os.rename(my_file, base + '.bin')
        plt.savefig('img_data-test/'+base+'.png')
        plt.show()
        plt.clf()

print("\n Show This message when the Conversion to Spectrographs are complete.")


In [ ]:
# Data preprocessing.****************************************************
# Read metadata file
metadata = pd.read_csv("./small Xeno-Canto/birdsong_metadata.csv")
header = list(metadata.head())


# Get bird names
bird_name = metadata['english_cname'].values
u, f = np.unique(bird_name, return_counts=True)
#print("Unique Bird Names  \n",u) # Dont touch this part - works

# Take 5 unique bird names for this project
uniq_birds = list(u[4:10]) + list(u[12:15])
data_train = []
data_test = []
y_train = []
y_test = []
bird_name_dict = {}
print(uniq_birds)


# Get file_id corresponding to bird names
for i in range(len(uniq_birds)) :
    df = metadata[metadata['english_cname'] == uniq_birds[i]]
    df = df['file_id'].values
    df = df.tolist()
    data_train.append(df[0])
    y_train.append(i)
    bird_name_dict[i] = uniq_birds[i]
    data_test += df[1:]
    y_test += [i] * (len(df) - 1)

In [ ]:
#print(os.listdir('./small Xeno-Canto/songs'))
songs = []                               # This array has all the bird songs. Still in .flac format.
directory = './small Xeno-Canto/songs/songs/'

for dirname, _, filenames in os.walk('./small Xeno-Canto/songs/songs/'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        songs.append(filename)
        #print(songs)


#print("Songs :...............")
#print(songs.pop(2))

print("data structure.")
print(metadata.head())
print(metadata.info(verbose=True))



In [ ]:
# CONVERTING BIRD SONGS WHICH ARE AUDIO FILES INTO IMAGE FILES --> iMAGES ARE SPECTROGRAPHS OF THE AUDIO DATA.
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
pathlib.Path('img_data/').mkdir(parents=True, exist_ok=True)

for dirname, _, filenames in os.walk('./small Xeno-Canto/songs/songs/'):
    for filename in filenames:
        #print(filename)
        songname = './small Xeno-Canto/songs/songs/'+filename  # Because you need the whole path.
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB')
        plt.axis('off')
        base = os.path.splitext(filename)[0]
        # os.rename(my_file, base + '.bin')
        plt.savefig('img_data/'+base+'.png')
        plt.show()
        plt.clf()


# We will listen to this file:
#

In [ ]:
# Trying to add labels now , from the csv files - pandas provides the df object.
from keras_preprocessing.image import ImageDataGenerator
tdf = pd.read_csv('./small Xeno-Canto/birdsong_metadata.csv')
#print(tdf.head())
#print(tdf.info(verbose=True))


# Cleanup train_df
tdf = tdf.drop(columns='who_provided_recording')
tdf = tdf.drop(columns='country')
tdf = tdf.drop(columns='latitude')
tdf = tdf.drop(columns='longitute')
tdf = tdf.drop(columns='license')
# Find a better way to drop multiple columns drop dataframe columns.



# For brevity and clarity in processing.
tdf['file_id'] = tdf['file_id'].apply(str)
# train_df = tdf  ----> According to proper processing procedures. The one at the bottom is a manual hack.
"""train_df = pd.read_csv('./small Xeno-Canto/traincsv.csv')
train_df.fillna(0, inplace=True)
test_df = pd.read_csv("./small Xeno-Canto/testcsv.csv")
test_df.fillna(0, inplace=True)
print("This one -------------------------------")
print(test_df.head())
print(test_df.info(verbose=True))
"""
#  HERE IS WHERE WE LEFT OFF --> 3.34PM. April 24th.
# Here is the fix --> 6th May , Friday.

print("\n  As far as I am concerned - This part is Complete.  ")

In [ ]:
import tensorflow as tf
print("\n  ",tf.__version__)
#import keras as K
print("\n " )

print("******************************  End of Processing. !!************************************************")